# Mathematical biology 
In this section, we will go through some of the applications in the mathematical modeling.


## 1. Lotka Voltera model
Also known as predator-prey model, it is useful in describing the dynamics of the population of two species which depend on each other. For example, foxes (predator) and rabbits (prey).

Mathematically, the dynamics of the population over time is given by a system of first-order non-linear ODEs:
$$ \frac{dx}{dt} = x(\alpha -\beta y)$$
$$ $$
$$ \frac{dy}{dt} = y(-\delta +\gamma x)$$
for $\alpha, \beta, \gamma, \delta > 0$

Here $x$ represents the population of the preying species while $y$ represents the population of the predators while $\frac{dx}{dt}$ and $\frac{dy}{dt}$ represents the changes in the populations.

In [4]:
import numpy as np

In [5]:
## code to compute derivatives

In [6]:
## populatioin dynamics plots

In [7]:
## phase plots

## 2. Epidemics predictions

One of the interesting class of models for predicting the dynamics of the contegious diseases is Compartmental models, where general population is assigned to either of Susceptible, Infectious or Recovered (SIR) compartments. People then progress between these compartments and the models try to appoximate how many people will be in a particular compartment as the diseases spread. 

In order to formulate the dynamics of the spread of the epidemic, we start with assumig S, I and R are functions of time and people in category S (susceptible) can get infected after being in contact with an I (infected) individual. It is also assumed that population in category I develops immunity to the disease and tehy leave the I category at a cure rate denoted by $\gamma$.

With the total population $N= S + I + R$ being constant, it is rather simple model with huge approximations. Assuming, on average, an individual S encounters $\beta$ individuals per time interval, and $\gamma$ individuals leaving the infected group.

Mathematically, dynamics can be captured by following system of equations:

$$
\begin{align}
\frac{dS}{dt} &=- \frac{\beta S}{N}I \\
\\
\frac{dI}{dt} &= \frac{\beta S}{N}I - \gamma I \\
\\
\frac{dR}{dt} &= \gamma I \\
\end{align}
$$




In [8]:
## define linspace and derivative

In [9]:
## population dynamics

In [10]:
## phase diagrams

In [ ]:
## further models overview